In [123]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [489]:
import pandas as pd

def f1(row, data, param):
    if param['state'] == '' or row[0].find('edit') > 0:
        param['state'] = row[0].split('[')[0]
    else:
        town = row[0].split('(')[0].strip()
        data.append([param['state'], town])
        
def get_list_of_university_towns():
    df = pd.read_csv('university_towns.txt',sep='\n', header=None)
    df['State']=''
    df['RegionName']=''

    for x in range(len(df[0])):
        if 'edit' in df[0][x]:
            state = df[0][x]
        else:
            df.State[x] = state

    df.State = df.State.apply(lambda x: x.replace('[edit]',''))
    df.RegionName = df[0].str.split('(').apply(lambda x: x[0])
    df.School = df[0].str.split('(').apply(lambda x: x[1:])
    df = df[df.State !='']
    df = df.drop([0],axis=1)
    return df
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
   #your code here

get_list_of_university_towns().head()

C:\Users\Acer\Anaconda\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo


In [301]:
#CLEAN DATA
df = pd.read_excel('gdplev.xls')

#drop useless columns
df.drop(df.columns[[0,1,2,3,5,7]], axis=1, inplace=True)
df = df.drop([0,1,2,3,4,5,6])

df.columns = ['Quarter','GDP']

df['Year'] = df.Quarter.str.split('q').apply(lambda x: int(x[0]))
df['GDP'] = pd.to_numeric(df['GDP'],errors='coerce')
df = df[df.Year>1999].reset_index().drop(columns='index')

df.head()

,Quarter,GDP,Year
0,2000q1,12359.1,2000
1,2000q2,12592.5,2000
2,2000q3,12607.7,2000
3,2000q4,12679.3,2000
4,2001q1,12643.3,2001


In [316]:
'''Returns the year and quarter of the recession start time as a string value in a format such as 2005q3'''
def get_recession_start(df):
    rec = 0
    quarter = df.Quarter[0]
    recession_start = []
    y = 1 
    while y in range(len(df.GDP)):
        if df.GDP[y-1] > df.GDP[y]:
            rec +=1
        else:
            quarter = df.Quarter[y]
            rec = 0
        if rec == 2:
            break
        y +=1
    return quarter
    
get_recession_start(df)

'2008q2'

In [345]:
'''Returns the year and quarter of the recession end time as a string value in a format such as 2005q3'''

def get_recession_end(df):
    y = df.index[df.Quarter == get_recession_start(df)].tolist()[0]
    quarter = df.Quarter[y-1]
    rec = 0
    recession_end = []
    while y in range(len(df.GDP)):
        if df.GDP[y-1] < df.GDP[y]:
            rec +=1
            quarter = df.Quarter[y]
        else:
            rec = 0
        if rec == 2:
            break
        y +=1
    return quarter
    #your code here

get_recession_end(df)


'2009q4'

In [356]:
'''Returns the year and quarter of the recession bottom time as a string value in a format such as 2005q3'''
def get_recession_bottom(df):
    start = df.index[df.Quarter == get_recession_start(df)].tolist()[0]
    end = df.index[df.Quarter == get_recession_end(df)].tolist()[0]
    df = df.loc[start:end]
    return df.Quarter[df.GDP == df.GDP.min()].tolist()[0]
    #your code here
    
get_recession_bottom(df) 

'2009q2'

In [486]:
def new_col_names():
    #generating the new coloumns names 
    years = list(range(2000,2017))
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    return quar_years[:67]

def convert_housing_data_to_quarters():
    #clean data
    df_homes = pd.read_csv('City_Zhvi_AllHomes.csv', encoding = "ISO-8859-1")

    df_homes = df_homes.replace({"State": states})
    df_homes = df_homes.drop(columns = ['RegionID','Metro','CountyName','SizeRank'])

    df_homes = df_homes.drop(df_homes.columns.to_series()["1996-04":"1999-12"], axis=1)
    df_homes = df_homes.drop(df_homes.columns.to_series()["2016-10":"2019-04"], axis=1)
    df_homes = df_homes.set_index(['RegionName','State'])
    df_homes = df_homes.dropna()

    #Create the new columns
    x = 0
    col=0
    length = len(df_homes.columns)
    while x < length:
        df_homes[new_col_names()[col]] = df_homes.iloc[:,[x,x+1,x+2]].mean(axis=1)
        x +=3
        col +=1

    a = list(range(0,201))

    df_homes = df_homes.drop(df_homes.columns[a],axis=1)
    
    return df_homes
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
RegionName,State,,,,,,,,,,,,,,,,,,,,,
New York,New York,220433.333333,228866.666667,236166.666667,242266.666667,248633.333333,256000.000000,265700.000000,277100.000000,285033.333333,286300.000000,...,502866.666667,513133.333333,522800.000000,5.329000e+05,5.431333e+05,5.540000e+05,5.624000e+05,5.696667e+05,5.761667e+05,5.840333e+05
Los Angeles,California,208166.666667,214900.000000,221466.666667,227766.666667,234566.666667,241333.333333,248200.000000,255833.333333,265500.000000,277000.000000,...,488200.000000,500666.666667,508900.000000,5.193000e+05,5.293667e+05,5.384000e+05,5.504667e+05,5.625000e+05,5.721667e+05,5.852333e+05
Houston,Texas,106266.666667,106300.000000,106233.333333,106500.000000,106600.000000,107033.333333,108066.666667,109466.666667,110533.333333,111300.000000,...,126600.000000,129400.000000,132500.000000,1.354333e+05,1.387333e+05,1.414333e+05,1.445333e+05,1.477000e+05,1.513000e+05,1.558667e+05
Chicago,Illinois,142233.333333,147433.333333,153033.333333,158700.000000,163566.666667,167033.333333,171433.333333,176833.333333,182366.666667,187033.333333,...,184833.333333,187733.333333,191233.333333,1.934333e+05,1.982333e+05,2.003333e+05,2.029667e+05,2.051333e+05,2.094000e+05,2.130333e+05
San Antonio,Texas,92133.333333,91800.000000,91133.333333,90800.000000,90300.000000,89700.000000,89433.333333,89400.000000,89500.000000,90000.000000,...,123200.000000,125366.666667,127066.666667,1.292000e+05,1.312000e+05,1.340000e+05,1.366000e+05,1.383333e+05,1.408000e+05,1.433667e+05
Philadelphia,Pennsylvania,51466.666667,52500.000000,53600.000000,54433.333333,55200.000000,55966.666667,56500.000000,57300.000000,58800.000000,60366.666667,...,104833.333333,106033.333333,107266.666667,1.076000e+05,1.091000e+05,1.103000e+05,1.107667e+05,1.118000e+05,1.157000e+05,1.208000e+05
Phoenix,Arizona,114500.000000,116700.000000,118800.000000,120933.333333,123366.666667,125266.666667,126600.000000,127900.000000,129500.000000,131133.333333,...,161566.666667,163366.666667,165566.666667,1.680667e+05,1.720000e+05,1.781000e+05,1.827667e+05,1.870333e+05,1.912667e+05,1.965667e+05
Las Vegas,Nevada,136933.333333,138266.666667,139100.000000,140333.333333,142333.333333,144600.000000,147333.333333,150800.000000,154600.000000,157266.666667,...,173033.333333,175966.666667,179000.000000,1.822000e+05,1.851000e+05,1.892000e+05,1.925000e+05,1.961667e+05,2.004667e+05,2.049000e+05
San Diego,California,224066.666667,235866.666667,246833.333333,256633.333333,266900.000000,277166.666667,286233.333333,295700.000000,306866.666667,319900.000000,...,474933.333333,481166.666667,487066.666667,4.924667e+05,5.013667e+05,5.117000e+05,5.205000e+05,5.319667e+05,5.382667e+05,5.447000e+05


In [499]:
#Not my code, Coppied this off of GitHub and tried to understand it

def run_ttest():
      
    df1 = get_list_of_university_towns()
    begin = get_recession_start(df)
    ending = get_recession_end(df)
    low = get_recession_bottom(df)
    house = convert_housing_data_to_quarters()
    
    prices_begin = house[begin]
    prices_end = house[ending]
    ratio = prices_begin.divide(prices_end)
        
    ratio_college = ratio[list(df1.index)].dropna()
    
    ratio_not_college_indices = set(house.index) - set(ratio_college.index)
    
    ratio_not_college = ratio.loc[list(ratio_not_college_indices)].dropna()
        
    statistic, p_value = tuple(ttest_ind(ratio_college, ratio_not_college))
    
    outcome = statistic < 0
    
    different = p_value < 0.01
    
    better = ["non-university town", "university town"]
    
    return (different, p_value, better[outcome])

    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
  #your code here

run_ttest()

C:\Users\Acer\Anaconda\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\Acer\Anaconda\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


(True, 2.5694629829509587e-32, 'non-university town')